# Data Wrangling: Join, Combine and Reshape

When data be spread across a number of files or datasets

Hierarchical indexing
Have multiple index levels on an axis

Work with higher dimensional data in a lower dimensional form


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data = pd.Series(
    np.random.uniform(size=9),
    index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], [1, 2, 3, 1, 3, 1, 2, 2, 3]],
)

data


In [ ]:
# A Series with MultiIndex
data.index


In [ ]:
# Partial indexing
data["b"]


In [ ]:
data["b":"c"]


In [ ]:
# Selection from "inner level", select all of the values that contains value 2 from the second index level
data.loc[:, 2]


In [ ]:
data.unstack()


In [ ]:
data.unstack().stack()


In [ ]:
# Either axis can have a hierarchical index
# Each index element have to have the same shape as the row or column
# Repeated index will be categorized
frame = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
    index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
    columns=[["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]],
)


In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]


In [ ]:
frame


In [ ]:
# See how many levels an index has
frame.index.nlevels


In [ ]:
frame["Ohio"]


In [ ]:
# A MultiIndex can be created by itself, then reused.
pd.MultiIndex.from_arrays(
    [["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]], names=["state", "color"]
)


### Reordering and sorting levels

Rearrange the order of the levels on the axis or sort the data by the values in one specific level.

Use the `swaplevel` method
Takes two level numbers or names and returns a new object with the levels interchanged


In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)


In [ ]:
frame.groupby(level="key2").sum()


In [ ]:
frame.sum(axis="columns")


In [ ]:
# Indexing with a DataFrame's columns
# Use one or more columns from a DataFrame as the row index, or move row index in to the DataFrame's columns

frame = pd.DataFrame(
    {
        "a": range(7),
        "b": range(7, 0, -1),
        "c": ["one", "one", "one", "one", "two", "two", "two"],
        "d": [1, 2, 3, 0, 1, 2, 3],
    }
)


In [ ]:
frame2 = frame.set_index(["c", "d"])


In [ ]:
# By default, columns are removed from the DataFrame, use drop=False to keep the index
frame2 = frame.set_index(["c", "d"], drop=False)


In [ ]:
# Does the opposite to the set_index, hierarchical index levels are moved into the columns
frame2.reset_index()


## 8.2 Combining and Merging Datasets

### pandas.merge

Connect rows in DataFrames based on one or more keys

Many-to-Many merge forms the Cartesian product of the matching keys.
All possible ordered paris will be combined

### pandas.concat

Concatenate or "stack" objects together along an axis

### combine_first

Splice overlapping data to fill in missing values in one object wit valyes from another


In [ ]:
df1 = pd.DataFrame(
    {
        "key": ["b", "b", "a", "c", "a", "a", "b"],
        "data1": pd.Series(range(7), dtype="int64"),
    }
)

df2 = pd.DataFrame(
    {
        "key": ["a", "b", "d", "b"],
        "data2": pd.Series(range(4), dtype="int64"),
    }
)

# Take df1, find items that is common between df1 and df2
# the unique element in both df will be dropped
# the overlapped element in both column will be added to new column
pd.merge(df1, df2, on="key")


In [ ]:
# Left , right, outer to decide if want to keep the missing values
pd.merge(df1, df2, on="key", how="outer")


In [ ]:
left = pd.DataFrame(
    {
        "key1": ["foo", "foo", "bar"],
        "key2": ["one", "two", "one"],
        "lval": pd.Series([1, 2, 3], dtype="Int64"),
    }
)

right = pd.DataFrame(
    {
        "key1": ["foo", "foo", "bar", "bar"],
        "key2": ["one", "one", "one", "two"],
        "rval": pd.Series([4, 5, 6, 7], dtype="Int64"),
    }
)

pd.merge(left, right, on=["key1", "key2"], how="outer")


In [ ]:
# Treatment of overlapping column names
# because both dataframe contains the "key2" column name
pd.merge(left, right, on=["key1"])

# Use suffixes to customize the overlapped column name
pd.merge(left, right, on=["key1"], suffixes=("_left", "_right"))


### Merging on index

Pass `left_index=True` or `right_index=True` to indicate that the index should be used as the merge key.


In [ ]:
left1 = pd.DataFrame(
    {"key": ["a", "b", "a", "a", "b", "c"], "value": pd.Series(range(6), dtype="int64")}
)

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

pd.merge(left1, right1, left_on="key", right_index=True, how="outer")


In [ ]:
# Joining with hierarchically indexed data
lefth = pd.DataFrame(
    {
        "key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
        "key2": [2000, 2001, 2002, 2001, 2002],
        "data": pd.Series(range(5), dtype="Int64"),
    }
)

righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002],
    ]
)

righth = pd.DataFrame(
    {
        "event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64", index=righth_index),
        "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64", index=righth_index),
    }
)

pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")


In [ ]:
left2 = pd.DataFrame(
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],
    index=["a", "c", "e"],
    columns=["Ohio", "Necada"],
).astype("Int64")

right2 = pd.DataFrame(
    [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0], [13.0, 14.0]],
    index=["b", "c", "d", "e"],
    columns=["Mussouri", "Alabama"],
).astype("Int64")

pd.merge(left2, right2, how="outer", left_index=True, right_index=True)
# Same as using DataFramels join method
left2.join(right2, how="outer")


## Concatenating Along an Axis

Concatenation or stacking.


In [ ]:
arr = np.arange(12).reshape((3, 4))
np.concatenate([arr, arr], axis=0)


In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")


In [ ]:
# By default, cerateing another series, vertically stack
pd.concat([s1, s2, s3], axis="index")

# Return a data frame
pd.concat([s1, s2, s3], axis="columns")

s4 = pd.concat([s1, s3])

pd.concat([s1, s4], axis="columns")


In [ ]:
# Create a hierarchical index on the concatenation axis
result = pd.concat([s1, s2, s3], keys=["one", "two", "three"])


In [ ]:
# When use columns as the axis, the keys become headers
result = pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])


In [ ]:
df1 = pd.DataFrame(
    np.arange(6).reshape((3, 2)), index=["a", "b", "c"], columns=["one", "two"]
)


df2 = pd.DataFrame(
    5 + np.arange(4).reshape((2, 2)), index=["a", "c"], columns=["three", "four"]
)

# keys argument is used to create a hierarchical index
# where the first level can be used to identify each of the concatenated DataFrame objects
pd.concat([df1, df2], keys=["level1", "level2"], axis="columns")

# Can also pass in a dictionary
pd.concat({"level1": df1, "level2": df2}, axis="columns")


In [ ]:
pd.merge(df1, df2, left_index=True, right_index=True, how="outer")


In [ ]:
# The last consideration concerns FataFrames in which the row index does not contain any relevant data.
# So the row index is to be ignored.

df1 = pd.DataFrame(np.random.standard_normal((3, 4)), columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)), columns=["b", "d", "a"])

# If directly concatenate, the index number will be messed up
pd.concat([df1, df2])

# to keep a clean index number
pd.concat([df1, df2], ignore_index=True)


## Combining Data with Overlap

Two datasets with indexes that overlap in full or in part.
An example of NumPy's `where` function, which performs the array-oriented equivalent of an if-else expression.


In [ ]:
a = pd.Series([np.nan, 2.5, 0, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"])

b = pd.Series([0, np.nan, 2, np.nan, np.nan, 5], index=["a", "b", "c", "d", "e", "f"])

# Whenever value in a are null, valyes from b are selected
# This doesn't check the input label are aligned or not
np.where(pd.isna(a), b, a)

# to line up values by index, use combine_first method
a.combine_first(b)


In [ ]:
df1 = pd.DataFrame(
    {"a": [1, np.nan, 5, np.nan], "b": [np.nan, 2, np.nan, 6], "c": range(2, 18, 4)}
)

df2 = pd.DataFrame({"a": [5, 4, np.nan, 3, 7], "b": [np.nan, 3, 4, 6, 8]})

df1.combine_first(df2)


# 8.3 Reslaping and Pivoting

## Reshaping with Hierarchical indexing

**stack** "Rotates" of pivots from the columns in the data to rows

**unstack** Pivots from the rows into the columns


In [ ]:
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)),
    index=pd.Index(["Ohio", "Calorado"], name="state"),
    columns=pd.Index(["one", "two", "three"], name="number"),
)

data


In [ ]:
data.stack()


In [ ]:
data.unstack(level=3)


In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64", name="one")

s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64", name="two")

data2 = pd.concat([s1, s2], keys=["one", "two"])


In [ ]:
data2.unstack()


In [ ]:
pd.merge(s1, s2, left_index=True, right_index=True, how="outer").T


In [ ]:
result = data.unstack()

df = pd.DataFrame(
    {"left": result, "right": result + 5},
    columns=pd.Index(["left", "right"], name="side"),
)

df

In [ ]:
df.unstack(level='state').stack(level='side')

## Pivoting "Long" to "Wide" format

long or stacked format - individual values are represented by a single row in a table.

In [ ]:
data = pd.read_csv("./datasets/macrodata.csv")

data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

data.head()

# use pandas.PeriodIndex (repersents time intervals)
# df.pop will return the column and delete it from dataframe
periods = pd.PeriodIndex(
    year=data.pop("year"), quarter=data.pop("quarter"), name="date"
)

periods


In [ ]:
data.index = periods.to_timestamp("D")


In [ ]:
data = data.reindex(columns=['realgdp', 'infl', 'unemp'])
data.columns.name = 'item'

In [ ]:
long_data = (data.stack().reset_index().rename(columns={0: 'value'}))

In [54]:
pivoted = long_data.pivot(index='date', columns='item', values='value')

In [55]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [56]:
long_data['value2'] = np.random.standard_normal(len(long_data))

In [61]:
# By omitting the last argument, obtain a DataFrame with hierarchial columns
pivoted = long_data.pivot(index='date', columns='item')

In [65]:
long_data

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.883439
1,1959-01-01,infl,0.000,-0.112430
2,1959-01-01,unemp,5.800,-0.983666
3,1959-04-01,realgdp,2778.801,1.968074
4,1959-04-01,infl,2.340,2.962557
...,...,...,...,...
604,2009-04-01,infl,3.370,0.114902
605,2009-04-01,unemp,9.200,0.313499
606,2009-07-01,realgdp,12990.341,0.042202
607,2009-07-01,infl,3.560,-1.403990


In [67]:
unstacked = long_data.set_index(['date', 'item']).unstack(level='item')
unstacked

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01  0.00   2710.349   5.8 -0.112430  0.883439 -0.983666
1959-04-01  2.34   2778.801   5.1  2.962557  1.968074 -0.727242
1959-07-01  2.74   2775.488   5.3  1.473527  0.290802  0.530519
1959-10-01  0.27   2785.204   5.6  0.784008  1.061694  1.831184
1960-01-01  2.31   2847.699   5.2 -0.190774  0.600811  0.881887
...          ...        ...   ...       ...       ...       ...
2008-07-01 -3.16  13324.600   6.0  0.513182  0.135714  0.432322
2008-10-01 -8.79  13141.920   6.9 -1.768465  0.075122  0.856034
2009-01-01  0.94  12925.410   8.1 -0.692090  1.407073  0.563715
2009-04-01  3.37  12901.504   9.2  0.114902 -1.312783  0.313499
2009-07-01  3.56  12990.341   9.6 -1.403990  0.042202  0.175207

[203 rows x 6 columns]

## Pivoting "Wide" to "Long" format
In inverse operation of pivot for DataFrames is pandas.melt

Merges multiple columns in to one

In [68]:
df = pd.DataFrame(
    {"key": ["foo", "bar", "baz"], "A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
)

melted = df.melt(id_vars="key")

# Use pivot to reshape the data back
melted.pivot(index='key', columns='variable', values='value')

# Specify the subset of columns to use as value columns
# id_vars are group identifiers
pd.melt(df, id_vars='key', value_vars=['A', 'B'])

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7
